Mandelbrot with 10k zoom levels using dask.delayed and dask.array.
- Lazily define each zoom level as a delayed task
- Stack as a dask.array with chunking
- Preview a few levels and save PNGs
- Optionally write entire stack to Zarr (out-of-core)

Import libraries and prepare an output folder. Zarr is optional.

In [1]:
import os
os.makedirs("outputs", exist_ok=True)

import numpy as np
import dask
    # noqa: F401
import dask.array as da
from dask import delayed
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt

try:
    import zarr  # noqa: F401
    HAS_ZARR = True
except Exception:
    HAS_ZARR = False

Set parameters: image size, number of levels, zoom, iterations, dtypes and chunking.

In [2]:
HEIGHT = 512
WIDTH = 512
N_LEVELS = 10_000
CENTER = (-0.75, 0.0)
SCALE0 = 3.0
ZOOM_PER_LEVEL = 1.03
MAX_ITER = 256
DTYPE = np.uint16
CHUNKS = (1, 256, 256)

Helper to convert iteration counts to uint8 for PNG saving.

In [3]:
def to_uint8(arr, max_val=MAX_ITER):
    a = np.asarray(arr, dtype=np.float32)
    a = np.clip(a, 0, float(max_val))
    a = (255.0 * a / float(max_val)).astype(np.uint8)
    return a

Define a small Mandelbrot renderer returning escape-time counts as a NumPy array.

In [4]:
def mandelbrot_array(height, width, center, scale, max_iter, dtype=np.uint16):
    cx, cy = center
    aspect = width / height
    x = np.linspace(cx - (scale * aspect) / 2.0, cx + (scale * aspect) / 2.0, width, dtype=np.float64)
    y = np.linspace(cy - scale / 2.0, cy + scale / 2.0, height, dtype=np.float64)
    C = x[None, :] + 1j * y[:, None]
    Z = np.zeros_like(C)
    counts = np.zeros(C.shape, dtype=np.int32)
    mask = np.ones(C.shape, dtype=bool)
    for k in range(max_iter):
        Z[mask] = Z[mask] * Z[mask] + C[mask]
        escaped = (np.abs(Z) > 2.0) & mask
        counts[escaped] = k
        mask &= ~escaped
        if not mask.any():
            break
    counts[mask] = max_iter
    return counts.astype(dtype, copy=False)

Wrap the renderer in a dask.delayed function and build a lazy 10k-level stack as a dask.array.

In [5]:
@delayed
def render_level(i, height, width, center, scale0, zoom_per_level, max_iter, dtype=np.uint16):
    scale = scale0 / (zoom_per_level ** i)
    return mandelbrot_array(height, width, center, scale, max_iter, dtype)

levels = [
    da.from_delayed(
        render_level(i, HEIGHT, WIDTH, CENTER, SCALE0, ZOOM_PER_LEVEL, MAX_ITER, DTYPE),
        shape=(HEIGHT, WIDTH), dtype=DTYPE
    )
    for i in range(N_LEVELS)
]
stack = da.stack(levels, axis=0).rechunk(CHUNKS)

Preview a few levels (first, middle, last), save a montage and individual PNGs.

In [6]:
idxs = [0, N_LEVELS // 2, N_LEVELS - 1]
with ProgressBar():
    sample_imgs = da.take(stack, idxs, axis=0).compute()

# Save montage
fig, axes = plt.subplots(1, len(idxs), figsize=(12, 4))
for ax, img, i in zip(axes, sample_imgs, idxs):
    ax.imshow(img, cmap="magma", origin="lower")
    ax.set_title(f"Level {i}")
    ax.axis("off")
fig.tight_layout()
fig.savefig("outputs/mandelbrot_preview_levels.png", dpi=150)
plt.close(fig)

# Save each as uint8 PNG
for img, i in zip(sample_imgs, idxs):
    img8 = to_uint8(img, MAX_ITER)
    plt.imsave(f"outputs/mandelbrot_level_{i}.png", img8, cmap="magma", origin="lower")

[                                        ] | 0% Completed | 226.68 us

[                                        ] | 0% Completed | 126.86 ms

[                                        ] | 0% Completed | 228.27 ms

[                                        ] | 0% Completed | 329.24 ms

[                                        ] | 0% Completed | 430.29 ms

[                                        ] | 0% Completed | 531.43 ms

[#############                           ] | 32% Completed | 632.40 ms

[#############                           ] | 32% Completed | 733.53 ms

[##########################              ] | 65% Completed | 834.37 ms

[########################################] | 100% Completed | 935.38 ms

Compute a single level on demand and save it as PNG.

In [7]:
level_to_compute = 1234
with ProgressBar():
    img_level = stack[level_to_compute].compute()
img8 = to_uint8(img_level, MAX_ITER)
plt.imsave(f"outputs/mandelbrot_level_{level_to_compute}.png", img8, cmap="magma", origin="lower")

[                                        ] | 0% Completed | 231.15 us

[                                        ] | 0% Completed | 344.63 ms

[                                        ] | 0% Completed | 481.44 ms

[                                        ] | 0% Completed | 582.22 ms

[                                        ] | 0% Completed | 683.05 ms

[                                        ] | 0% Completed | 783.84 ms

[                                        ] | 0% Completed | 884.64 ms

[########################################] | 100% Completed | 985.45 ms

Optionally write the entire 10k-level stack to a Zarr store on disk (out-of-core). This may take time and disk space.

In [8]:
if HAS_ZARR:
    zarr_path = "outputs/mandelbrot_10k.zarr"
    with ProgressBar():
        stack.to_zarr(zarr_path, overwrite=True)
else:
    pass  # Zarr not installed; skipping write.